In [1]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

c:\Users\Osmar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('test.csv')

print(train_data.head())
print(test_data.head())

             ID                                           Sentence  \
0    library#01  All material that is stored in the repository ...   
1  library#02-1  The Library may want to accept important digit...   
2  library#02-2  The Library may want to accept important digit...   
3    library#03  Once material has arrived, <referential>it</re...   
4    library#04  Allows resources to be reviewed before a decis...   

  Detected as        Resolved as  
0     NOCUOUS                NaN  
1     NOCUOUS                NaN  
2   INNOCUOUS  digital materials  
3   INNOCUOUS           material  
4   INNOCUOUS          resources  
              ID                                           Sentence  \
0  diagnostic#01  The absolute time of input will be associated ...   
1  diagnostic#02  Each sensed data item will be qualified as to ...   
2  diagnostic#03  Sensed data input to the tool shall be validat...   
3  diagnostic#04  The faults identified and <referential>their</...   
4  diagnostic#

In [3]:

# Preprocesamiento de datos
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
texts = train_data["Sentence"].tolist()
labels = [0 if label == "innocuous" else 1 for label in train_data["Detected as"]]

# Tokenización
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}, self.labels[idx]

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(encodings, labels)

# División del conjunto de entrenamiento y validación
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

# Cargar modelo con las capas de clasificación
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Configurar los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Entrenamiento
trainer.train()

# Guardar el modelo entrenado
model.save_pretrained("./modelo_ambiguo")
tokenizer.save_pretrained("./modelo_ambiguo")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Osmar\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/21 [00:00<?, ?it/s]

TypeError: vars() argument must have __dict__ attribute

In [ ]:

test_texts = test_data["Sentence"].tolist()
test_labels = [0 if label == "innocuous" else 1 for label in test_data["Detected as"]]

test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings["input_ids"]),
    torch.tensor(test_labels)
)

# Predicción
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted_labels))